# Infer Nuclick on DB points for phase 2

In [ ]:
from histomics.engine.inference.nuclick.nuclick_annotation_pipe import NuclickAnnotationPipe
from histomics.data.db.backup import backup
from loguru import logger
import matplotlib.pyplot as plt

DB_PATH = "sqlite:////home/owkin/project/database/first_phase.db"
annot_pipe = NuclickAnnotationPipe(root_path=DB_PATH, nuclick_version="unet")

# Select tiles from Phase 2 only with pulled annotations of points
SQL_QUERY_PHASE_2 = ("SELECT tiles.tile_id,"
    " tiles.coords,tiles.slide_name,slides.abstra_slide_path FROM slides"
    " INNER JOIN tiles ON tiles.slide_name=slides.slide_name"
    " WHERE tiles.annotation_type = 'point'"
    " AND tiles.nuclick_has_been_inferred = False"
    " AND tiles.is_pulled_from_cytomine=1 AND tiles.timestamp_pushed >= '2024-02-08'"
)


SQL_QUERY_PHASE_1 = ("SELECT tiles.tile_id,"
    " tiles.coords,tiles.slide_name,slides.abstra_slide_path FROM slides"
    " INNER JOIN tiles ON tiles.slide_name=slides.slide_name"
    " WHERE tiles.annotation_type = 'point'"
    " AND tiles.nuclick_has_been_inferred = False"
    " AND tiles.is_pulled_from_cytomine=1 AND tiles.timestamp_pushed <= '2024-01-01'"
)


dataset = annot_pipe.get_inference_dataset(db_root_path=DB_PATH, sql_query=SQL_QUERY_PHASE_1)

In [ ]:
dataset.dataset.dataframe

In [ ]:
list_all_tile_ids_phase_2 = dataset.dataset.dataframe.index.tolist().copy()

chunk_size = 20

chunks = [list_all_tile_ids_phase_2[chunk_size*i:chunk_size*(i+1)] for i in range(len(list_all_tile_ids_phase_2)//chunk_size + 1)]

In [ ]:
len(list_all_tile_ids_phase_2)

In [ ]:
len(chunks)

In [ ]:
BEST_MODEL_PATH = (
    "/home/owkin/project/experiments/nuclick/"
    "consep_pcns_train_unet_nuclick/checkpoint_epoch35.pth"
)

model = annot_pipe.get_nuclick_model(nuclick_version="unet", weights_file_path=BEST_MODEL_PATH)

## Before Launching Inference

In [ ]:
# Backup

backup("first_phase")

In [ ]:
chunks[1][6]

In [ ]:
# These tiles create CUDA OOM errors because they have a lot of instances (more than 500)
TILES_EXCLUDE = [71932813, 61093044]

In [ ]:
annot_pipe.set_inferer(model, model_full_name="nuclick_unet_consep_pcns_35_epochs")

for idx_chunk, tile_id_chunk in enumerate(chunks):
    # DONT LAUNCH IT YET
    logger.info(f"Processing chunk: {idx_chunk}")

    subset = [str(x) for x in tile_id_chunk if x not in TILES_EXCLUDE]  # Transform to string
    annot_pipe.set_dataset(sql_query=SQL_QUERY_PHASE_1, subset_tiles_ids=subset)
    nuclick_results = annot_pipe.get_nuclick_results()

    annot_pipe.dump_results_viz(nuclick_results, dump_directory="/home/owkin/project/database/nuclick_preannotations_05_04")
    plt.close("all")
    logger.warning("Uploading results to the database")
    annot_pipe.upload_to_database(nuclick_results)

In [ ]:
all_nuclick_results

In [ ]:
all_nuclick_results[0]["70397741"]["instance_map"]

In [ ]:
all_nuclick_results[0]["70397741"]

In [ ]:
annot_pipe.dump_results_viz(all_nuclick_results[0], dump_directory="/home/owkin/project/database/nuclick_preannotations_04_04")